# Web Scraping

Es una técnica que nos permite "raspar" o "arañar" la información que hay en las páginas web. De manera que todo lo que puedes ver y leer en cualquier página web también puedes extraerlo de manera automatizada con python (con algunas excepciones).

En este notebook vamos a ver una de las maneras de recuperar esta información usando las librerías request, para extraer, y bs4 (BeautifulSoup) para darle forma. 


## Introducción a `requests.Response`
Cuando haces una solicitud HTTP con `requests.get(url)`, el resultado es un objeto `Response` que contiene:

 - Código de estado (status_code): Indica si la solicitud fue exitosa (200), redirigida (301), no encontrada (404), etc.
 - Cabeceras (headers): Metadatos sobre la respuesta (tipo de contenido, servidor, etc.).
 - Contenido (text o content): El HTML de la página web.

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Hacemos llamada a la web

https://es.wikipedia.org/wiki/Anexo:Libros_m%C3%A1s_vendidos

In [32]:
# url de la que queremos extraer la información
url = 'https://es.wikipedia.org/wiki/Anexo:Libros_m%C3%A1s_vendidos'

In [33]:
# Hacer la solicitud HTTP
response = requests.get(url)
response

<Response [200]>

Los tipos de respuestas que nos podemos encontrar cuando navegamos por internet: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

### Analizamos la respuesta a nuestra llamada

In [7]:
response.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="es" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Anexo:Libros más vendidos - Wikipedia, la enciclopedia libre</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-l

In [9]:
# Mostrar el código de estado
print(f"Código de estado: {response.status_code}")

# Mostrar las cabeceras de la respuesta
print("Cabeceras:")
for key, value in response.headers.items():
    print(f"{key}: {value}")

# Mostrar los primeros 500 caracteres del HTML
print("\nHTML (primeros 500 caracteres):")
print(response.text[:500])


Código de estado: 200
Cabeceras:
date: Thu, 20 Feb 2025 17:24:15 GMT
vary: Accept-Encoding,Cookie,Authorization
server: ATS/9.2.6
x-content-type-options: nosniff
content-language: es
accept-ch: 
last-modified: Tue, 18 Feb 2025 21:54:41 GMT
content-type: text/html; charset=UTF-8
content-encoding: gzip
age: 59536
x-cache: cp6014 hit, cp6015 hit/6
x-cache-status: hit-front
server-timing: cache;desc="hit-front", host;desc="cp6015"
strict-transport-security: max-age=106384710; includeSubDomains; preload
report-to: { "group": "wm_nel", "max_age": 604800, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }
nel: { "report_to": "wm_nel", "max_age": 604800, "failure_fraction": 0.05, "success_fraction": 0.0}
set-cookie: WMF-Last-Access=21-Feb-2025;Path=/;HttpOnly;secure;Expires=Tue, 25 Mar 2025 00:00:00 GMT, WMF-Last-Access-Global=21-Feb-2025;Path=/;Domain=.wikipedia.org;HttpOnly;secure

### Extraemos los datos de la página

En concreto nos interesa la tabla con los libros más leídos.

Para desenmarañara la respuesta de request vamos a usar la librería beautifulsoup, ya importada anteriormente

In [104]:
# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Encontrar la primera tabla en la página
tabla = soup.find("table", {"class": "wikitable"})
# tabla = soup.find("table")
len(tabla)


2

In [35]:
# Extraer las filas de la tabla
filas = tabla.find_all("tr")
filas

[<tr>
 <th>Libro</th>
 <th>Autor</th>
 <th>Idioma original</th>
 <th>Primera edición</th>
 <th>Ventas aproximadas
 </th></tr>,
 <tr>
 <td><i><a href="/wiki/Don_Quijote_de_la_Mancha" title="Don Quijote de la Mancha">Don Quijote</a></i></td>
 <td><a href="/wiki/Miguel_de_Cervantes" title="Miguel de Cervantes">Miguel de Cervantes</a></td>
 <td><a href="/wiki/Idioma_espa%C3%B1ol" title="Idioma español">Español</a></td>
 <td>1605</td>
 <td>Más de 500 millones
 </td></tr>,
 <tr>
 <td><i><a href="/wiki/Historia_de_dos_ciudades_(novela)" title="Historia de dos ciudades (novela)">Historia de dos ciudades</a></i> (<i>A Tale of Two Cities</i>)</td>
 <td><a href="/wiki/Charles_Dickens" title="Charles Dickens">Charles Dickens</a></td>
 <td><a href="/wiki/Idioma_ingl%C3%A9s" title="Idioma inglés">Inglés</a></td>
 <td>1859</td>
 <td>Más de 200 millones<sup class="reference separada" id="cite_ref-3"><a href="#cite_note-3"><span class="corchete-llamada">[</span>3<span class="corchete-llamada">]</span><

In [37]:
# Lista para guardar los datos
data = []

# Iterar sobre las filas (omitir la primera si son encabezados)
for fila in filas[1:]:  
    columnas = fila.find_all("td")  
    if columnas:  
        # Extraer los valores de las columnas y limpiar texto
        datos_fila = [columna.text.strip() for columna in columnas]
        data.append(datos_fila)

# Convertir a DataFrame
# columnas = ["Libro", "Autor", "Ventas estimadas", "Idioma original", "Año de publicación"]
df = pd.DataFrame(data)

# Mostrar la tabla
df.head()


,0,1,2,3,4
0,Don Quijote,Miguel de Cervantes,Español,1605,Más de 500 millones
1,Historia de dos ciudades (A Tale of Two Cities),Charles Dickens,Inglés,1859,Más de 200 millones[3]​
2,El Señor de los Anillos (The Lord of the Rings),J. R. R. Tolkien,Inglés,1954,150 millones[4]​[5]​
3,Harry Potter y la piedra filosofal (Harry Pott...,J.K. Rowling,Inglés,1997,140 millones[6]​
4,El principito (Le Petit Prince),Antoine de Saint-Exupéry,Francés,1943,140 millones[7]​


### Extraer los encabezados originales

In [50]:
encabezados = [th.text.strip() for th in tabla.find_all("th")]
encabezados

['Libro', 'Autor', 'Idioma original', 'Primera edición', 'Ventas aproximadas']

In [51]:
df.columns = encabezados
df

,Libro,Autor,Idioma original,Primera edición,Ventas aproximadas
0,Don Quijote,Miguel de Cervantes,Español,1605,Más de 500 millones
1,Historia de dos ciudades (A Tale of Two Cities),Charles Dickens,Inglés,1859,Más de 200 millones[3]​
2,El Señor de los Anillos (The Lord of the Rings),J. R. R. Tolkien,Inglés,1954,150 millones[4]​[5]​
3,Harry Potter y la piedra filosofal (Harry Pott...,J.K. Rowling,Inglés,1997,140 millones[6]​
4,El principito (Le Petit Prince),Antoine de Saint-Exupéry,Francés,1943,140 millones[7]​
5,El hobbit (The Hobbit),J. R. R. Tolkien,Inglés,1937,Más de 100 millones[8]​
6,"Sueño en el pabellón rojo (红楼梦, Hóng lóu mèng)",Cao Xueqin,Chino,1759–1791,Más de 100 millones[9]​
7,"Triple representatividad (论三个代表, Sānge dàibiǎo)",Jiang Zemin,Chino,2001,100 millones[10]​[Nota 1]​
8,Diez negritos,Agatha Christie,Inglés,1939,100 millones[11]​
9,Las aventuras de Alicia en el país de las mara...,Lewis Carroll,Inglés,1865,Más de 100 millones[12]​


###  Diferencia entre `<tr>` ,`<th>` y `<td>`

Estos símbolos hacen referencia a table row, table header y table data

| Etiqueta | ¿Qué representa? | Ejemplo en una tabla |
|----------|----------------|----------------------|
| `<tr>`   | **Una fila completa** de la tabla | `<tr> ... </tr>` |
| `<th>`   | **Celda de encabezado** (títulos) | `<th>Libro</th>` |
| `<td>`   | **Una celda dentro de la fila** | `<td>Don Quijote</td>` |



### Extraemos el resto de las tablas

Como vemos en la web desde el explorador hay más de una tabla, vamos a extraerlas todas.

Si buscamos usando `find_all` en vez de `find`, nos encuentra todas las tablas, no solo la primera. Vemos que hay 6 en total.

In [114]:
tabla_all = soup.find_all("table")
len(tabla_all)

6

In [128]:
# Encontrar todas las tablas
tablas = soup.find_all("table", {"class": "wikitable"})

# Lista para almacenar los DataFrames
dataframes = {}

# Iterar sobre cada tabla y convertirla en un DataFrame
for i, tabla in enumerate(tablas):
    # Obtener encabezados
    headers = [th.text.strip() for th in tabla.find_all("th")]
    
    # Obtener filas de la tabla
    filas = []
    for fila in tabla.find_all("tr")[1:]:  # Omitimos la primera fila si son encabezados
        columnas = fila.find_all("td")
        datos_fila = [columna.text.strip() for columna in columnas]
        if datos_fila:  # Evitar filas vacías
            filas.append(datos_fila)
    
    # Crear DataFrame
    df = pd.DataFrame(filas, columns=headers if headers else None)
    
    # Guardar en el diccionario
    dataframes[f"tabla_{i+1}"] = df



In [134]:
dataframes['tabla_1'].head()

,Libro,Autor,Idioma original,Primera edición,Ventas aproximadas
0,Don Quijote,Miguel de Cervantes,Español,1605,Más de 500 millones
1,Historia de dos ciudades (A Tale of Two Cities),Charles Dickens,Inglés,1859,Más de 200 millones[3]​
2,El Señor de los Anillos (The Lord of the Rings),J. R. R. Tolkien,Inglés,1954,150 millones[4]​[5]​
3,Harry Potter y la piedra filosofal (Harry Pott...,J.K. Rowling,Inglés,1997,140 millones[6]​
4,El principito (Le Petit Prince),Antoine de Saint-Exupéry,Francés,1943,140 millones[7]​


In [135]:
dataframes['tabla_2'].head()

,Libro,Autor,Idioma original,Primera edición,Ventas aproximadas
0,"El león, la bruja y el armario o El león, la b...",C. S. Lewis,Inglés,1950,85 millones[13]​
1,Ella (She),Henry Rider Haggard,Inglés,1887,83 millones[14]​
2,El código Da Vinci (The Da Vinci Code),Dan Brown,Inglés,2003,80 millones[15]​
3,Harry Potter y la cámara secreta (Harry Potter...,J.K. Rowling,Inglés,1998,77 millones[16]​
4,Harry Potter y el prisionero de Azkaban (Harry...,J.K. Rowling,Inglés,1999,65 millones[16]​


In [143]:
primer_titulo = soup.find_all("p")

# Extraer todos los párrafos antes de la primera sección
texto_inicio = []
for p in primer_titulo.find_previous_siblings():
    if p.name == "p":  # Solo tomamos párrafos de texto
        texto_inicio.append(p.get_text())

# Invertir el orden para mantener el flujo correcto
texto_inicio = "\n".join(texto_inicio[::-1])

# Mostrar el texto extraído
print(texto_inicio)

AttributeError: ResultSet object has no attribute 'find_previous_siblings'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

### Otros objetos que podemos extraer de una página HTML

| **Etiqueta**  | **Descripción** | **Ejemplo** |
|--------------|---------------|------------|
| `<html>`  | Contenedor principal de una página web | `<html>...</html>` |
| `<head>`  | Contiene metadatos y enlaces a estilos o scripts | `<head>...</head>` |
| `<title>` | Define el título de la página (aparece en la pestaña del navegador) | `<title>Mi Página</title>` |
| `<body>`  | Contiene el contenido visible de la página | `<body>...</body>` |
| `<h1>` - `<h6>` | Encabezados de diferentes niveles (h1 es el más grande) | `<h1>Título principal</h1>` |
| `<p>`     | Párrafo de texto | `<p>Hola, este es un párrafo.</p>` |
| `<br>`    | Salto de línea | `Hola<br>mundo` → (Hola en una línea, mundo en la siguiente) |
| `<hr>`    | Línea horizontal divisoria | `<hr>` |
| `<a>`     | Enlace a otra página o sitio web | `<a href="https://example.com">Visitar</a>` |
| `<img>`   | Imagen dentro de la página | `<img src="imagen.jpg" alt="Descripción">` |
| `<ul>`    | Lista desordenada (con viñetas) | `<ul><li>Elemento</li></ul>` |
| `<ol>`    | Lista ordenada (numerada) | `<ol><li>Primero</li><li>Segundo</li></ol>` |
| `<li>`    | Elemento de una lista (`<ul>` o `<ol>`) | `<li>Elemento de lista</li>` |
| `<table>` | Crea una tabla | `<table>...</table>` |
| `<tr>`    | Fila de una tabla | `<tr>...</tr>` |
| `<th>`    | Celda de encabezado de tabla | `<th>Título</th>` |
| `<td>`    | Celda normal de una tabla | `<td>Dato</td>` |
| `<form>`  | Formulario de entrada de datos | `<form>...</form>` |
| `<input>` | Campo de entrada en un formulario | `<input type="text">` |
| `<button>` | Botón interactivo | `<button>Enviar</button>` |
| `<div>`   | Contenedor genérico en bloque | `<div>Contenido</div>` |
| `<span>`  | Contenedor genérico en línea | `<span>Texto</span>` |
| `<strong>` | Texto en negrita | `<strong>Importante</strong>` |
| `<em>`    | Texto en cursiva (énfasis) | `<em>Destacado</em>` |


#### Títulos y encabezados

In [151]:
primer_titulo = soup.find_all("h1")
primer_titulo

[<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-namespace">Anexo</span><span class="mw-page-title-separator">:</span><span class="mw-page-title-main">Libros más vendidos</span></h1>]

In [152]:
primer_titulo = soup.find_all("h2")
primer_titulo

[<h2 class="vector-pinnable-header-label">Contenidos</h2>,
 <h2 id="Más_de_cien_millones_de_ejemplares"><span id="M.C3.A1s_de_cien_millones_de_ejemplares"></span>Más de cien millones de ejemplares</h2>,
 <h2 id="Entre_cincuenta_y_cien_millones_de_ejemplares">Entre cincuenta y cien millones de ejemplares</h2>,
 <h2 id="Entre_treinta_y_cincuenta_millones_de_ejemplares">Entre treinta y cincuenta millones de ejemplares</h2>,
 <h2 id="Entre_veinte_y_treinta_millones_de_ejemplares">Entre veinte y treinta millones de ejemplares</h2>,
 <h2 id="Entre_diez_y_veinte_millones_de_ejemplares">Entre diez y veinte millones de ejemplares</h2>,
 <h2 id="Véase_también"><span id="V.C3.A9ase_tambi.C3.A9n"></span>Véase también</h2>,
 <h2 id="Notas">Notas</h2>,
 <h2 id="Referencias">Referencias</h2>,
 <h2 id="Enlaces_externos">Enlaces externos</h2>]

In [153]:
primer_titulo = soup.find_all("p")
primer_titulo

[<p>Este anexo provee una lista de los <b>libros más vendidos</b> en cualquier idioma. El concepto de «más vendido» (que generalmente se expresa mediante los términos <a href="/wiki/Superventas" title="Superventas">superventas</a> o <i>best-seller</i>) se refiere al número de copias (o a una estimación del mismo) que se han vendido de cada libro, y no a las que fueron impresas o distribuidas gratuitamente. No se incluyen en esta lista ni <a class="mw-redirect" href="/wiki/C%C3%B3mic" title="Cómic">cómics</a>, ni <a href="/wiki/Libro_de_texto" title="Libro de texto">libros de texto</a>. Tampoco aparecen obras políticas, como las de <a href="/wiki/Mao_Zedong" title="Mao Zedong">Mao Zedong</a>, ni religiosas, como la <i><a href="/wiki/Biblia" title="Biblia">Biblia</a></i> (la obra más vendida de la historia, pero con ventas e impresiones muy inexactas) el <i><a href="/wiki/Cor%C3%A1n" title="Corán">Corán</a></i>, el <i><a href="/wiki/Libro_de_Morm%C3%B3n" title="Libro de Mormón">Libro de 

## Diferencias entre Selenium y Requests  

Ambas herramientas se utilizan para **Web Scraping**, pero tienen enfoques muy diferentes.  

| **Característica**     | **Selenium** 🖥️ | **Requests** 🌐 |
|----------------------|-----------------|----------------|
| **Modo de funcionamiento** | Simula un navegador real | Envía solicitudes HTTP |
| **Carga JavaScript** | ✅ Sí | ❌ No |
| **Interacción con la web** | ✅ Puede hacer clic, desplazarse y completar formularios | ❌ Solo obtiene HTML estático |
| **Velocidad** | 🐢 Más lento (abre un navegador) | 🚀 Más rápido |
| **Bloqueos por el sitio** | ❌ Fácilmente detectado por los sitios web | ✅ Menos propenso a bloqueos |
| **Uso de recursos** | 🔋 Alto consumo de CPU y RAM | 🔋 Bajo consumo |
| **Casos de uso ideales** | Scraping de sitios con contenido dinámico (JavaScript) o interacción con formularios | Scraping de sitios con HTML estático (sin JavaScript) |

####  ¿Cuándo usar Selenium?
- Si el sitio **usa JavaScript** para cargar contenido.  
- Cuando necesitas **interactuar** con la web (clics, scroll, inputs, etc.).  
- Para **automatizar pruebas de software** en sitios web.  

####  ¿Cuándo usar Requests?
- Si el contenido **estático** se puede obtener directamente en el HTML.  
- Cuando necesitas **eficiencia y rapidez**.  
- Si el sitio **no requiere interacciones** complejas.  

####  ¿Cuál es mejor?
Depende del caso. Si el sitio es estático, usa **Requests**.  
Si es dinámico con JavaScript, usa **Selenium**   


## Introducción a Selenium

Importamos la libreria `selenium`, la cual a diferencia de `requests` abre el explorador a través de `webdriver`. El objetivo principal es nuevamente obtener una tabla con datos para luego hacer un análisis exploratorio de los precios y sus productos.

Para ello vamos a tener que entrar en las categorías de productos e ir recolectando la información a partir de ahí.



In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

`links_elements` contiene todos los elementos de tipo `<a>` anchor (enlaces). Al ejecutar este código se crea el set() links. Que contiene todos los links subyacentes a partir de la url establecida, es decir nuestro _web scraper_ va a entrar en todos y cada uno de los enlaces que hay a partir de nuestra url, y va a comprobar que no estén vacíos antes de guardarlos en el set.

In [38]:
# Configurar el WebDriver
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()

# Acceder a la web de Makro
driver.get("https://www.makro.es/productos")
time.sleep(5)  # Esperar a que la página cargue completamente

links_elements = driver.find_elements(By.TAG_NAME, "a")

# Lista para guardar los enlaces únicos
links = set()

# Recorrer los elementos y obtener el texto y el href
for link in links_elements:
    try:
        url = link.get_attribute("href")
        if url:  # Solo guardar si el enlace no está vacío
            links.add(url)
    except:
        continue

# Cerrar el navegador
driver.quit()

In [39]:
len(links_elements)

226

In [40]:
links

{'https://cih.makro.es/es/support/tickets/new',
 'https://es.linkedin.com/company/makro-autoservicio-mayorista-sa',
 'https://maps.google.com/maps/?q=40.358334,-3.824555',
 'https://talento.makro.es/',
 'https://tienda.makro.es/shop/promotions',
 'https://tienda.makro.es/shop/storelist/829911ee-f923-408d-9f37-167fa36d542e',
 'https://tienda.makro.es/shop/storelist/c266ad5d-a0ff-4a17-8b24-528ff7e42016',
 'https://twitter.com/makroesp',
 'https://www.facebook.com/MakroEspana',
 'https://www.instagram.com/makroesp/',
 'https://www.makro.es/',
 'https://www.makro.es/MCW/ES_Makro/Home/info%20y%20servicios/productos/carnes',
 'https://www.makro.es/MCW/ES_Makro/Home/info-y-servicios/productos/cestas-navidad',
 'https://www.makro.es/aviso-legal',
 'https://www.makro.es/bases',
 'https://www.makro.es/calidad-y-seguridad',
 'https://www.makro.es/compliance',
 'https://www.makro.es/compliance#canalInterno',
 'https://www.makro.es/compra-como-quieras',
 'https://www.makro.es/compra-como-quieras/ca

No todos los enlaces nos interesan. Por ejemplo los términos y condiciones, los servicios, los enlaces a youtube... Así que vamos a filtrar por aquellos que nos interesan.

Creamos una lista de palabras clave y sólo retendremos los links que contienen estas palabras, que son los productos alimenticios principalmente. Por lo tanto creamos una lista con las palabras clave que orienten nuestra búsqueda a nuestros intereses.

In [41]:
palabras_clave = ['carne','pescado','fruta','verdura','marisco','vino','congelado']
# clases_articulos = ['amplificad']
articulos = []
for clases in palabras_clave:
    for link in sorted(links):  # Ordenar alfabéticamente
        if clases in link:
            
            articulos.append(link)
articulos = set(articulos)
print(len(articulos))

46


In [42]:
articulos

{'https://www.makro.es/MCW/ES_Makro/Home/info%20y%20servicios/productos/carnes',
 'https://www.makro.es/productos/congelados',
 'https://www.makro.es/productos/congelados/aperitivos',
 'https://www.makro.es/productos/congelados/carnes',
 'https://www.makro.es/productos/congelados/carnes/caza',
 'https://www.makro.es/productos/congelados/carnes/preparados',
 'https://www.makro.es/productos/congelados/fruta-verdura',
 'https://www.makro.es/productos/congelados/mariscos',
 'https://www.makro.es/productos/congelados/pan',
 'https://www.makro.es/productos/congelados/patatas',
 'https://www.makro.es/productos/congelados/pescado',
 'https://www.makro.es/productos/congelados/postres',
 'https://www.makro.es/productos/congelados/postres/helados',
 'https://www.makro.es/productos/congelados/precocinados',
 'https://www.makro.es/productos/frutas',
 'https://www.makro.es/productos/frutas/citricos',
 'https://www.makro.es/productos/frutas/citricos/naranja',
 'https://www.makro.es/productos/frutas/e

Hasta aquí hemos trabajado con Selenium, que nos es más intuitivo para obtener el listado de enlaces a partir de los cuales vamos a recabar la información.

A partir de ahora vamos a usar requests, que es más difícil de detectar

In [59]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
}
df = pd.DataFrame([])
for url in articulos:
    print(url)
    # Realizar la solicitud HTTP
    response = requests.get(url, headers=headers)
    time.sleep(3) 
    try:
        soup = BeautifulSoup(response.text, "html.parser")
    except:
        print(response.status_code)


https://www.makro.es/productos/vinos/ribera
https://www.makro.es/productos/mariscos
https://www.makro.es/productos/frutas/citricos
https://www.makro.es/productos/mariscos/langosta
https://www.makro.es/productos/vinos/delecto
https://www.makro.es/productos/vinos/selecion-sumiller
https://www.makro.es/productos/congelados/carnes/caza
https://www.makro.es/productos/pescados/pescado-azul
https://www.makro.es/productos/vinos/vinos-tintos
https://www.makro.es/productos/congelados/postres/helados
https://www.makro.es/productos/congelados/fruta-verdura
https://www.makro.es/productos/vinos/vinos-blancos
https://www.makro.es/productos/vinos/sumiller-makro
https://www.makro.es/productos/congelados/pan
https://www.makro.es/productos/vinos/gran-monumento
https://www.makro.es/productos/congelados/aperitivos
https://www.makro.es/productos/vinos/rioja
https://www.makro.es/productos/congelados/patatas
https://www.makro.es/productos/verduras/setas
https://www.makro.es/productos/verduras/tomate
https://w

`BeautifulSoup` nos facilita la interpretación del HTML obtenido en el `.get`. El objetivo a partir de ahora sería encontrar a ojo, cómo se llaman las clases producto, descripciuón, precio, imagen...

Pero como se puede observar en la última línea del siguiente cuadro nos han pillado y no hemos conseguido entrar en el marketplace.

In [62]:
soup

<!DOCTYPE html>
<html><head><meta content="width=device-width,initial-scale=1" name="viewport"/><title>403</title></head><body style="background:#f2f5f8;margin:0"><div style="padding:20px;background:#002d72"><svg class="m" height="24" style="display:table" viewbox="0 0 136 25" width="129"><path d="M18.1333932,25 L23.3476299,9.30188577 L23.3476299,25 L31.1301731,25 L31.1301731,0.00765476126 L20.6240762,0.00765476126 L15.5650866,12.5038274 L10.5065456,0.00765476126 L0,0.00765476126 L0,25 L7.78299198,25 L7.78299198,9.30188577 L13.0358161,25 L18.1333932,25 Z M101.772514,15.2843068 C104.20038,15.0893356 106.621067,13.4309541 106.621067,10.7076602 L106.621067,5.08411232 L106.621067,4.53747231 C106.339289,1.72097044 104.233583,0.200374633 101.737516,0 L81.7169288,0 L81.7173775,25 L89.4994721,25 L89.4994721,15.2843068 L92.597234,15.2843068 L97.6553262,25 L106.605363,25 L100.581688,15.2843068 L101.772514,15.2843068 Z M99.6170027,8.80162461 C99.6170027,9.34241098 99.4828442,9.81700618 98.5154666

En el diccionario headers podemos incluir algunas opciones para tratar de engañar al servidor de destino y hacerle creer que somos un usuario entrando a través de un explorador normal.

Aunque en este caso tampoco funciona. Otro método que ayuda a saltar las defensas del servidor web es añadir una pausa como hemos hecho anteriormente con `time.sleep(5)`

A continuación abrimos el notebook `Web_scraping_thomann` donde sí hemos conseguido entrar y podremos ver el proceso completo. Intenta crear un dataframe con la información de los productos thomann para luego analizarlos.

In [65]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
    "Referer": "https://www.google.com/",
    "DNT": "1",  # Do Not Track
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "max-age=0",
}
response = requests.get('https://www.makro.es/productos/carnes', headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
soup

<!DOCTYPE html>
<html><head><meta content="width=device-width,initial-scale=1" name="viewport"/><title>403</title></head><body style="background:#f2f5f8;margin:0"><div style="padding:20px;background:#002d72"><svg class="m" height="24" style="display:table" viewbox="0 0 136 25" width="129"><path d="M18.1333932,25 L23.3476299,9.30188577 L23.3476299,25 L31.1301731,25 L31.1301731,0.00765476126 L20.6240762,0.00765476126 L15.5650866,12.5038274 L10.5065456,0.00765476126 L0,0.00765476126 L0,25 L7.78299198,25 L7.78299198,9.30188577 L13.0358161,25 L18.1333932,25 Z M101.772514,15.2843068 C104.20038,15.0893356 106.621067,13.4309541 106.621067,10.7076602 L106.621067,5.08411232 L106.621067,4.53747231 C106.339289,1.72097044 104.233583,0.200374633 101.737516,0 L81.7169288,0 L81.7173775,25 L89.4994721,25 L89.4994721,15.2843068 L92.597234,15.2843068 L97.6553262,25 L106.605363,25 L100.581688,15.2843068 L101.772514,15.2843068 Z M99.6170027,8.80162461 C99.6170027,9.34241098 99.4828442,9.81700618 98.5154666